In [0]:
%scala

val containerName = "inbound"
val storageAcoountName = "storageaccountsala443"
val sas = "?sv=2020-08-04&ss=bfqt&srt=sco&sp=rwdlacupx&se=2022-02-25T02:59:43Z&st=2022-02-24T12:59:43Z&spr=https&sig=KO86N2LQwk81i3GKYRwqJHiLVQR24cPRtKN%2FsKT3BF4%3D"

val url = "wasbs://" + containerName + "@" + storageAcoountName + ".blob.core.windows.net"
var config = "fs.azure.sas." + containerName + "." + storageAcoountName + ".blob.core.windows.net"

dbutils.fs.unmount("/mnt/inbound")

dbutils.fs.mount(
	source = url,
	mountPoint = "/mnt/inbound",
extraConfigs = Map(config -> sas))

In [0]:
%scala
import org.apache.spark.sql.functions.get_json_object

In [0]:
%scala

val inboundFile = "dbfs:/mnt/inbound/B3_range*.csv"
val df = spark.read.csv(inboundFile)
df.createOrReplaceTempView("data_")


inboundFile: String = dbfs:/mnt/inbound/B3_range*.csv
df: org.apache.spark.sql.DataFrame = [_c0: string, _c1: string ... 1 more field]

In [0]:
%python
data = spark.sql("select * from data_")
data = data.toPandas()
# data

In [0]:
%python
spark.catalog.dropTempView("data_")
import re 
import pandas as pd
   
data['Ticket'] = [str(re.findall(r'\w{4}\d+',data['_c2'][x])).strip("[|]|''") for x,d in data.iterrows()]
data['Cod'] = [str(re.findall(r'\w{4}\d+',data['_c2'][x])).lstrip("[|''")[:4] for x,d in data.iterrows()]
data['Segmento'] = [str(re.findall(r'\w{5}\s\d|Tradicional|Novo Mercado',data['_c2'][x])).strip("]|[|''") for x,d in data.iterrows()]
data['char'] = [str(re.findall(r'TAG ALONG\\n\d{1,3}\W\sON\\n\d{1,3}\W\sPN|TAG ALONG\\n\d{1,3}\W\sON',data['_c2'][x])).strip("]|[|''") for x,d in data.iterrows()]
data['T_A_Ordinarias'] = [str(re.findall(r'\d{1,3}\W\sON',data['char'][x])).strip("[|]").strip("]|[|''")  for x,d in data.iterrows()]
data['T_A_Preferenciais'] = [str(re.findall(r'\d{1,3}\W\sPN',data['char'][x])).strip("]|[|''")  for x,d in data.iterrows()]
data['charFF'] = [str(re.findall(r'FREE FLOAT\\n\d{1,3}\W\d{1,3}\W\sON\\n\d+\W\d+\W\sPN|FREE FLOAT\\n\d{1,3}\W\d{1,3}\W\sON',data['_c2'][x])).strip("]|[|''") for x,d in data.iterrows()]
data['Free_Float_Ordinarias'] = [str(re.findall(r'\d{1,3}\W\d{1,3}\W\sON',data['charFF'][x])).strip("]|[|''")  for x,d in data.iterrows()]
data['Free_Float_Preferenciais'] = [str(re.findall(r'\d{1,3}\W\d{1,3}\W\sPN',data['charFF'][x])).strip("]|[|''")  for x,d in data.iterrows()]
data['Governanca'] = [str(re.findall(r'GOVERNANÇA\\nCEO\\n(.+)\\nSEGMENTO DE LISTAGEM',data['_c2'][x])).strip("[|]|'")  for x,d in data.iterrows()]
data['Socios_Majoritarios'] = [str(re.findall(r'SÓCIOS MAJORITÁRIOS\\n(.+)|SÓCIO MAJORITÁRIO\\n(.+)',data['_c2'][x])).strip("[|]|'|(|)")  for x,d in data.iterrows()]
data['Governo'] = [str(re.findall(r'GOVERNO',data['_c2'][x])).strip("[|]|'") for x,d in data.iterrows()]

In [0]:
%python
from datetime import datetime as dt
time = dt.now()
time = str(time)[:16]
data["Bronze_Load_Date"] = time
data = data.drop(columns=['_c0','_c1','_c2','char','charFF'])[2:]


In [0]:
%python
sparkdf = spark.createDataFrame(data)
display(sparkdf)



Ticket,Cod,Segmento,T_A_Ordinarias,T_A_Preferenciais,Free_Float_Ordinarias,Free_Float_Preferenciais,Governanca,Socios_Majoritarios,Governo,Bronze_Load_Date
OPCT3,OPCT,Novo Mercado,100% ON,,"34,80% ON",,Flavio Nogueira Pinheiro de Andrade,"""Flavio Nogueira Pinheiro de Andrade\\nDYNA II FUNDO DE INVESTIMENTO EM PARTICIPAÇÕES MULTIESTRATÉGIA\\nEquitas Adminsitradora de Fundos de Investimentos\\n30% ON\\n14% ON\\n5% ON\\nVer todos os sócios (31) →', '"",",,2022-02-24 14:41
CARD3,CARD,Novo Mercado,100% ON,,"44,20% ON",,Marcos Ribeiro Leite,"""Greeneville Delaware LLC.\\nGreeneville Delaware LLC\\n37% ON\\n17% ON\\nVer todos os sócios (4) →', '"",",,2022-02-24 14:41
CRIV3,CRIV,Tradicional,80% ON,0% PN,"8,56% ON","35,81% PN",Fábio Alberto Amorosino,"""Alfa Holdings S.A.\\nConsórcio Alfa de Administração S.A.\\nCorumbal Participações e Administração Ltda.\\n30% ON\\n30% ON\\n29% ON\\n0,03% PN\\n0,03% PN\\n21% PN\\nVer todos os sócios (7) →', '"",",,2022-02-24 14:41
HBOR3,HBOR,Novo Mercado,100% ON,,"43,78% ON",,Henry Borenstein,"""Hélio Borenstein S.A. - Administração, Participações e Comércio\\nHenrique Borenstein\\nTávola Capital\\n39% ON\\n7% ON\\n5% ON\\nVer todos os sócios (4) →', '"",",,2022-02-24 14:41
AFLT3,AFLT,Tradicional,80% ON,,"1,37% ON",,Fabiano Uchoas Ribeiro,"""Neoenergia S.A\\nIberdrola Energia S.A.\\n90% ON\\n9% ON\\nVer todos os sócios (2) →', '"",",,2022-02-24 14:41
PDTC3,PDTC,Novo Mercado,100% ON,,"17,03% ON",,Carlos Raimar Schoeninger,"""Fundação CPqD - Centro de Pesquisa e Desenvolvimento em Telecomunicações\\nBNDES Participações S.A. - BNDESPAR\\nLMC Brazil, LLC\\n55% ON\\n23% ON\\n5% ON\\nGOVERNO\\nVer todos os sócios (3) →', '"",",GOVERNO,2022-02-24 14:41
MDNE3,MDNE,Novo Mercado,100% ON,,"31,55% ON",,Carlos Roberto Barreto Gentil Filho,"""GUSTAVO JOSE MOURA DUBEUX\\nALUISIO JOSE MOURA DUBEUX\\nMARCOS JOSE MOURA DUBEUX\\n12% ON\\n11% ON\\n10% ON\\nVer todos os sócios (9) →', '"",",,2022-02-24 14:41
APER3,APER,Novo Mercado,100% ON,,"40,24% ON",,Mariana Maduro,"BC Gestão de Recursos Ltda. (""""Brasil Capital"""")\\nCredit Suisse Securities (Europe) Ltd.\\nPatria Investiments\\n18% ON\\n14% ON\\n9% ON\\nVer todos os sócios (6) →\'',",,2022-02-24 14:41
HBTS3,HBTS,Tradicional,,,,,SERGIO LUIZ COTRIM RIBAS,", ""CIA. Comercial de Imóveis\\n100% ON\\n36% PN\\nVer todos os sócios (3) →', '""",,2022-02-24 14:41
MTSA3,MTSA,Tradicional,80% ON,0% PN,,,Edvaldo Angelo,,,2022-02-24 14:41


In [0]:
%sql
create database if not exists bronze_dim


In [0]:
%python
# sparkdf = spark.createDataFrame(data)
sparkdf.createOrReplaceTempView("temp")




In [0]:
%sql
select distinct * from temp


In [0]:
%python
sparkdf.write.mode("overwrite").saveAsTable("bronze_dim.dim_categoria")

In [0]:
%scala
// HERE STARTS THE SETORIAL FILE ETL
val inboundFile = "dbfs:/mnt/inbound/DIM_SETORIAL.csv"
// val df = spark.read.csv(inboundFile).option("encoding", "utf8")
var df2 = spark.read.format("csv")
                    .option("delimiter", ",")
                    .option("header",false)
                    .option("encoding", "ISO-8859-1")
                    .load(inboundFile)
                            
df2.createOrReplaceTempView("data_")

In [0]:
%python
from pyspark.sql import functions as F

result = sqlContext.sql("""select _c5 as Cod_Listagem, _c3 as Setor from data_ """)

# result = result.withColumn('Bronze_Loaded_Date',F.current_timestamp())

# result = result.select("Cod_Listagem","Setor","Bronze_Loaded_Date").distinct().collect()

result = result.withColumn('Bronze_Loaded_Date',F.current_timestamp()).select("Cod_Listagem","Setor","Bronze_Loaded_Date").distinct()

# type(result)
display(result)
# del result




In [0]:
%python

result.write.mode("overwrite").saveAsTable("bronze_dim.dim_setores")

In [0]:
%scala
// #HERE STARTS THE FACT INGESTION

val inboundFACTfile = "dbfs:/mnt/inbound/fundamentus*.csv"
// val df = spark.read.csv(inboundFile).option("encoding", "utf8")
var df_FACT = spark.read.format("csv")
                    .option("delimiter", ",")
                    .option("header",false)
                    .option("encoding", "UTF-8")
                    .load(inboundFACTfile)
                            
df_FACT.createOrReplaceTempView("tempFACTfile")

inboundFACTfile: String = dbfs:/mnt/inbound/fundamentus*.csv
df_FACT: org.apache.spark.sql.DataFrame = [_c0: string, _c1: string ... 22 more fields]

In [0]:
%python
# DF_Spark = 
from pyspark.sql import functions as F

DF_Spark = sqlContext.sql("""select _c1 as Papel, _c2 as Cotacao,_c3 as P_L,_c4 as P_VP,_c5 as PSR,_c6 as Div_Yield, _c7 as P_Ativo,
_c8 as P_Capital_de_giro, _c9 as P_EBIT, _c10 as P_Ativ_Circ_Liq, _c11 as EV_EBIT, _c12 as EV_EBITDA,
_c13 as Mrg_Ebit, _c14 as Mrg_Liq, _c15 as LiqCorr, _c16 as ROIC, _c17 as ROE, _c18 as Liq2meses,
_c19 as PatrimLiq, _c20 as DivBrut_Patrim, _c21 as CrescRec5a, cast(left(_c23,10) as date) as inbound_date from tempFACTfile where _c1 <> 'Papel' """)

DF_Spark = DF_Spark.withColumn('Bronze_Loaded_Date',F.current_timestamp())

DF_Spark.write.mode("overwrite").saveAsTable("bronze_dim.Fato_B3")



